In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.4 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow_io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 65.8 MB/s eta 0:00:00


In [ ]:
! pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import imutils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA, KernelPCA
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_io as tfio
import keras_tuner as kt
import os
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Datasets/data.zip > /dev/null

In [ ]:
dir_cataract ='/content/Untitled Folder/cataract'
dir_normal ='/content/Untitled Folder/normal'
if not os.path.isdir(dir_cataract):
  os.makedirs(dir_cataract)
if not os.path.isdir(dir_normal):
  os.makedirs(dir_normal)
for root, dirs, files in os.walk((os.path.normpath('/content/data/')), topdown=False):
    for name in files:
        SourceFolder = os.path.join(root,name)
        if "normal" in name:
            shutil.move(SourceFolder, dir_normal)
        elif "cataract" in name :
            shutil.move(SourceFolder, dir_cataract)

In [ ]:
img_height = 256
img_width = 256

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
train_ds = image_dataset_from_directory('/content/Untitled Folder', 
                                        seed = 7, 
                                        image_size=(img_height, img_width), 
                                        validation_split=0.2, 
                                        subset='training')

Found 2922 files belonging to 2 classes.
Using 2338 files for training.


In [ ]:
val_ds = image_dataset_from_directory('/content/Untitled Folder', 
                                      seed = 7, 
                                      image_size=(img_height, img_width), 
                                      validation_split=0.2, 
                                      subset='validation')

Found 2922 files belonging to 2 classes.
Using 584 files for validation.


In [ ]:
data_augmentation = keras.Sequential(
    [
      layers.RandomFlip("horizontal_and_vertical"),
      layers.RandomRotation(factor=(-0.2, 0.2), fill_mode="constant"),
      layers.RandomContrast(0.2),
    ]
)

In [ ]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

In [ ]:
conv = tf.keras.applications.vgg16.VGG16(include_top=False)

conv.trainable = True
for layer in conv.layers[:-4]:
  layer.trainable = False

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
simple_cnn = tf.keras.models.load_model('/content/model.h5')

In [ ]:
def simpe_cnn_with_normalize_3():
  input = keras.Input(shape=(256, 256, 3))
  x = data_augmentation(input)
  x = keras.applications.vgg16.preprocess_input(x)
  x = conv(x)
  x = layers.Flatten()(x)
  x = layers.Dense(256, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  output = layers.Dense(1, activation="sigmoid")(x)
  return keras.Model(input, output)

In [ ]:
batch_size = 128
epochs = 1
tf.random.set_seed = 48

cnn_model = simpe_cnn_with_normalize_3()

cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="convent_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

cnn_model.fit(
    train_ds,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=val_ds,
)

74/74 [==============================] - 1808s 24s/step - loss: 0.6847 - accuracy: 0.7429 - val_loss: 0.6749 - val_accuracy: 0.7654


In [ ]:
cnn_model.save("model.h5")

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from glob import glob
import pandas as pd

img_size = (256, 256)
d = {0:'cataract', 1:'normal'}

def predict_images(dir='test_images', res_file='result.csv'):

    files = glob(dir+'/*')
    d_test={}
    for file in files:
        image = load_img(file, target_size=img_size)
        image = np.array(image)
        pred = simple_cnn.predict(np.expand_dims(image, axis=0))[0][0]
        pred = int(np.round(pred))
        d_test[file.split('/')[3]]=d[pred]
    
    d_csv = {'name':list(d_test.keys()), 'label': list(d_test.values())}
    df = pd.DataFrame.from_dict(d_csv)
    print(df)
    return df.to_csv(res_file, index=False)

In [ ]:
predict_images(dir='/content/test_images', res_file='result.csv')

1/1 [==============================] - 0s 490ms/step
    name   label
0  2.jpg  normal
1  3.jpg  normal
2  4.jpg  normal
3  1.jpg  normal


In [ ]:
import numpy as np
trainableParams = np.sum([np.prod(v.get_shape()) for v in cnn_model.trainable_weights])
nonTrainableParams = np.sum([np.prod(v.get_shape()) for v in cnn_model.non_trainable_weights])
totalParams = trainableParams + nonTrainableParams

print(trainableParams)
print(nonTrainableParams)
print(totalParams)

15468545
7635264
23103809


In [ ]:
import pandas as pd
# result = pd.read_csv('/content/result.csv')
result = pd.DataFrame({'name': ['1.jpg', '2.jpeg', '3.jpeg', '4.jpg'], 'label': ['normal', 'cataract', 'normal', 'cataract']})
# ref = pd.read_csv('/content/ref.csv')
ref = pd.DataFrame({'name': ['1.jpg', '2.jpeg', '3.jpeg', '4.jpg'], 'true_label': ['cataract', 'normal', 'normal', 'cataract']})
merg = pd.merge(ref, result, on='name', how='left')

from sklearn.metrics import classification_report
print(classification_report(y_true=merg.true_label, y_pred=merg.label))

              precision    recall  f1-score   support

    cataract       0.50      0.50      0.50         2
      normal       0.50      0.50      0.50         2

    accuracy                           0.50         4
   macro avg       0.50      0.50      0.50         4
weighted avg       0.50      0.50      0.50         4

